1. Parse https://rusdisinfo.voxukraine.org/

In [47]:
import time
import threading
import validators
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [48]:
url = "https://rusdisinfo.voxukraine.org/"

In [49]:
def get_links(url):
    options = Options()
    options.add_argument("--headless")

    driver = webdriver.Safari(options=options)
    driver.get(url)

    page = driver.page_source
    soup = BeautifulSoup(''.join(page), 'html.parser')

    titles = soup.find_all('a')
    links = []
    categories = []

    start = "/narratives/"
    for title in titles:
        if title["href"].startswith(start):
            links.append(title["href"])
            # correct title format
            categories.append(title.text.replace("# ", "").replace(",", "").replace(" ", "_").lower())
    
    links_df = pd.DataFrame(columns=categories)

    for i, link in enumerate(links):
        driver.get(url + link)
        sub_page = driver.page_source
        sub_soup = BeautifulSoup(''.join(sub_page), 'html.parser')

        headers = sub_soup.find_all('h3', class_='Narrative_fakeLink___YbTe')

        fakes_count = 0
        for j, header in enumerate(headers):
            driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div[2]/h3[{j+1}]').click()

            sub_page = driver.page_source
            sub_soup = BeautifulSoup(''.join(sub_page), 'html.parser')
            fakes = sub_soup.find_all('a', target='_blank')

            for k, fake in enumerate(fakes):
                links_df.loc[fakes_count+k, categories[i]] = fake["href"]
            fakes_count = fakes_count + len(fakes)
            driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div[2]/h3[{j+1}]').click()
    
    driver.quit()
    
    return links_df.fillna("")

In [50]:
df = get_links(url)

In [51]:
df

,maidan_in_2014_was_a_coup_d'état,nazis_and_ultra-nationalists_in_ukraine,nord_stream-2,russia_needs_to_protect_itself,ukraine_is_an_illiberal_state,ukraine_is_a_failed_state,the_west_controls_ukraine_and_uses_it_to_its_advantage,the_west_is_not_interested_in_dealing_with_ukraine_moreover_solving_its_problems.,russians_are_discriminated_in_ukraine,russia_is_not_an_agressor_towards_ukraine,ukraine_is_conducting_an_aggressive_policy,schismatic_ukrainian_church,russia_is_not_involved_in_mh17_crash;_ukraine_shot_down_the_plane
0,https://it.sputniknews.com/20211209/putin-quel...,https://uncutnews.ch/die-nato-rueckt-in-der-uk...,https://resetitaliablog.altervista.org/bloombe...,https://de.rt.com/europa/128476-ukraine-wird-i...,https://de.rt.com/meinung/117718-ukraine-wie-d...,https://www.controinformazione.info/crocevia-p...,https://www.imolaoggi.it/2018/12/06/ucraina-po...,https://uncutnews.ch/putin-an-biden-finnlandis...,https://de.news-front.info/2022/01/17/ukraine-...,https://de.rt.com/europa/128523-ukraine-als-au...,https://www.anti-spiegel.ru/2021/ukraine-updat...,http://www.antikrieg.com/aktuell/2018_12_17_uk...,https://de.rt.com/meinung/117472-investigative...
1,https://www.anti-spiegel.ru/2021/krieg-im-donb...,https://www.radioradio.it/2020/08/bieloRussia-...,https://www.corrierenazionale.net/il-prezzo-de...,https://de.rt.com/europa/128523-ukraine-als-au...,https://www.anti-spiegel.ru/2020/die-hagia-sop...,https://de.news-front.info/2022/01/02/politisc...,https://de.rt.com/europa/118309-voelkisches-ge...,https://uncutnews.ch/polen-und-die-ukraine-nic...,https://de.rt.com/europa/113926-ukraine-zensur...,https://de.rt.com/international/128829-trotz-k...,https://snanews.de/20211217/eu-verlaengert-san...,https://comedonchisciotte.org/mosca-costantino...,https://de.rt.com/meinung/107857-russland-verl...
2,https://de.rt.com/meinung/129862-lebkuchenhexe...,https://uncutnews.ch/die-us-kriegsplaene-fuer-...,https://www.corrierenazionale.net/il-prezzo-de...,https://de.news-front.info/2022/01/02/pentagon...,https://de.rt.com/europa/113926-ukraine-zensur...,https://comedonchisciotte.org/chi-vuole-un-po-...,https://www.unzensuriert.at/content/0029346-uk...,https://uncutnews.ch/aktuelles-von-der-ukraini...,https://de.rt.com/europa/113926-ukraine-zensur...,https://de.rt.com/2353/international/nato-und-...,https://uncutnews.ch/das-russische-fernsehen-u...,https://www.anti-spiegel.ru/2020/die-hagia-sop...,https://de.rt.com/meinung/106908-fall-nawalny-...
3,https://www.wsws.org/de/articles/2021/04/23/uk...,https://it.sputniknews.com/20170224/Maidan-naz...,https://www.controinformazione.info/leuropa-pa...,https://de.news-front.info/2022/01/02/pentagon...,https://de.rt.com/europa/110540-naziverehrung-...,https://www.ilfoglio.it/esteri/2016/02/17/news...,https://uncutnews.ch/warum-ermitteln-die-usa-g...,https://uncutnews.ch/warum-ist-die-ukraine-in-...,https://de.rt.com/europa/87515-ukrainisch-nach...,https://de.rt.com/2353/international/nato-und-...,https://de.rt.com/europa/129217-krieg-im-donba...,https://de.rt.com/europa/86860-rote-karte-fuer...,https://de.rt.com/meinung/95837-investigativer...
4,https://de.rt.com/meinung/128478-russland-und-...,https://it.sputniknews.com/20211210/ucraina-se...,https://it.sputniknews.com/20220103/lallarme-s...,https://snanews.de/20220116/peskow-nato-keine-...,https://www.anti-spiegel.ru/2021/wie-die-ukrai...,https://www.ilfoglio.it/esteri/2016/02/17/news...,https://uncutnews.ch/polen-und-die-ukraine-nic...,https://uncutnews.ch/warum-ist-die-ukraine-in-...,http://rt.de/,https://it.sputniknews.com/20211212/il-g7-cond...,https://de.rt.com/europa/115364-donbass-unter-...,https://de.rt.com/europa/84265-jw-timoschenko-...,https://de.rt.com/international/88772-malaysia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,,,,,,,,,,https://report24.news/situation-im-donbass-esk...,,,
125,,,,,,,,,,http://www.vita.it/it/article/2018/10/15/quell...,,,
126,,,,,,,,,,https://www.farodiroma.it/putin-la-russofobia-

In [72]:
def parse(link):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Safari(options=options)
    
    driver.execute_script(f"window.location.href = '{link}'")
    
    time.sleep(3)
    
    page = driver.page_source
    soup = BeautifulSoup(''.join(page), 'html.parser')

    articles = soup.find_all('p')
    articles_text = []

    for article in articles:
        articles_text.append(article.text)

    driver.quit()
    return articles_text

In [73]:
for col in df.columns:
    df[col] = df[col].apply(parse)

JavascriptException: Message: A JavaScript exception occured: Invalid URL
